In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../..")

In [4]:
from torchganime.models.ganime import GANime
from torchganime.data.dataloader.video import VideoData

/usr/local/lib/python3.9/dist-packages/horovod/common/util.py:258: UserWarning: Framework pytorch installed with version 1.13.0+cu117 but found version 1.13.1+cu117.
             This can result in unexpected behavior including runtime errors.
             Reinstall Horovod using `pip install --no-cache-dir` to build with the new version.
  warnings.warn(get_version_mismatch_message(name, version, installed_version))


In [37]:
dataloader = VideoData(
    train_paths="/TorchGANime/data/kny/raw_videos/01.mkv",
    val_paths="/TorchGANime/data/kny/raw_videos/01.mkv",
    batch_size=1,
    num_workers=1,
    image_size=[64, 128],
    min_max_len=[15, 20],
    duplicate_metric="mae",
    duplicate_threshold=0.01,
    root_dir="/TorchGANime/data/.cache",
    device="cpu",
    n_jobs=1,
    default_mode="video",
)

2023-03-04 11:31:01.063 | INFO     | scenedataset.scenedataset:_get_video_paths:550 - Found 1 videos


Removing duplicates from scene /TorchGANime/data/kny/raw_videos/01.mkv...:   0%|          | 0/527 [00:00<?, ?i…

2023-03-04 11:31:01.085 | INFO     | scenedataset.scenedataset:save_scenes:378 - Scene list file /TorchGANime/data/.cache/content_15_15/duplicate_metric_mae/threshold_0.01/_TorchGANime_data_kny_raw_videos_01.mkv.json already up to date.
2023-03-04 11:31:01.443 | INFO     | scenedataset.scenedataset:_get_video_paths:550 - Found 1 videos


Removing duplicates from scene /TorchGANime/data/kny/raw_videos/01.mkv...:   0%|          | 0/527 [00:00<?, ?i…

2023-03-04 11:31:01.465 | INFO     | scenedataset.scenedataset:save_scenes:378 - Scene list file /TorchGANime/data/.cache/content_15_15/duplicate_metric_mae/threshold_0.01/_TorchGANime_data_kny_raw_videos_01.mkv.json already up to date.


In [38]:
for batch in dataloader.train_dataloader():
    break

In [39]:
batch[1].shape

torch.Size([1, 3, 18, 64, 128])

In [40]:
batch[0]["current_frames"].shape

torch.Size([1, 3, 64, 128])

In [41]:
model = GANime(
    learning_rate=2.5e-4,
    vqgan_ckpt_path="/TorchGANime/checkpoints/vqgan_full_64x128/checkpoints/checkpoint.ckpt",
    vocab_size=1024,
    n_positions=1024,
    n_embd=768,
    n_layer=12,
    n_head=12,
    use_token_type_ids=False,
    rec_loss_weight=0.0,
    perceptual_loss_weight=0.0,
    mode="video",
)


Working with z of shape (1, 512, 32, 32) = 524288 dimensions.
loaded pretrained LPIPS loss from /TorchGANime/torchganime/models/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
loaded pretrained LPIPS loss from /TorchGANime/torchganime/models/lpips/vgg.pth


In [42]:
output = model(batch[0], batch[1])

In [43]:
output["prediction"].shape

torch.Size([1, 3, 18, 64, 128])

In [44]:
output.keys()

dict_keys(['prediction', 'loss', 'scce_loss', 'nll_loss', 'rec_loss', 'p_loss'])

In [45]:
output["loss"]

tensor(6.2488, grad_fn=<AddBackward0>)